In [1]:
import os
import sys
import logging
import itertools
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from typing import Dict, Any, List, Generator

In [2]:
# package imports
from vsim import electorate, candidates, simulation, voting_system

# Analysing proportional voting systems

We'll be using the case study of the Swedish parlimentary election systems, namely that there exists n-parties, voters vote for the preferred option (using the distance method we described earlier). The difference is the "winner" is not as well defined. 

In [3]:
# setup helper for generating experiments (basically all sets of params to run through sim)
def generate_experiment_matrix(experiments: Dict[str, List[Any]]):
    return (dict(zip(experiments, x)) for x in itertools.product(*experiments.values()))

columns=[
    "electorate_size",
    "clusters",
    "cluster_std" 
    "issues",
    "candidates",
    "electoral_system",
    "apathy_prob",
    "fairness"
]

experiments = {
    "electorate_size": [10_000],
    "clusters": [10],
    "cluster_std": [1],
    "electoral_system": ["proportional"],
    "apathy_prob": [0.0],
    "candidates": [2],
    "issues": [2],
}

records = []
columns = [*experiments.keys(), "fairness"]
experiment_matrix = list(generate_experiment_matrix(experiments))


# outer monte carlo sim
for i in range(1000):
    for params in experiment_matrix:

        # setup objects
        voters = electorate.setup_electorate(**params)
        parties = candidates.setup_candidates(**params, electorate=voters)
        system = voting_system.setup_voting_system(name=params["electoral_system"], **params)

        # simulate elections
        sim = simulation.VotingSimulator(electorate=voters, candidates=parties, system=system, log=logging.getLogger())

        result = sim.run()
        result_record = { "fairness": round(result.measured_fairness, 3), **params }
        result_record = {k: v for k, v in result_record.items() if k in columns}
        records.append(result_record)

RuntimeError: allocated_seats={0: 174, 1: 174}, off by more than one, only 348/349

In [ ]:
results_df = pd.DataFrame.from_records(records)
results_df.head()

In [ ]:
results_df.cumsum("fairness")

##### Store data or read from already performed experiment files

In [3]:
filepath = "data/fairness-convergence-f6da8c03-dce6-4604-90a7-b1778e1d1bdd.csv"